In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
%matplotlib inline

# Data files full clean

In [2]:
import os
import re

if not os.path.exists("./cleandata"):
    os.makedirs("./cleandata")
content = os.listdir("./newdata")

for item in content:
    # delete the text files and the urls files to clean the library
    #print(item)
    if item.find(".txt") > 0:
        # print(item)
        with open("./newdata/{}".format(item), "r", encoding="iso-8859-1") as f:
            lines = f.readlines()
        # if (os.path.exists("./cleandata/{}".format(item))):
        with open("./cleandata/{}".format(item), "w+") as f:

            for line in lines:
                if "10 Messungen" in line.strip("\n"):
                    line = line[line.find("Ch") :]
                    # print(newline)
                # if (line.strip("\n") != "þStartfischer1 p33-6 hf8 fc6x 0.9km")
                if "Ch" in line.strip("\n") or "V" in line.strip("\n"):
                    f.write(line.replace(";", ""))
f.close()

# print(content)

# Extract the Data and convert it to Dataframes and Excel files

In [4]:
import xlwt
a={}
lst={}
content = os.listdir("./cleandata")
result=pd.DataFrame()
verhaeltnis=pd.DataFrame()
deltas=pd.DataFrame()
for x in content:
    if (x.find(".txt") > 0 and x != ".git"):
        #print(x)
        df = pd.read_csv("./cleandata/{}".format(x),names=["channal", "min", "max", "x", "y"],sep="\s+",encoding="ISO-8859-1")
        dx = df.drop(["x", "y"], axis=1)
        dx["min"] = dx["min"]  # .map(lambda x: x.lstrip("+-;").rstrip("aAbBcC;"))
        dx["channal"] = dx["channal"]  # .map(lambda x: x.lstrip(";").rstrip(";"))
        if 100 in dx.index:
            dx = dx.drop(100)
        dx["min"] = pd.to_numeric(dx["min"])
        dx["max"] = pd.to_numeric(dx["max"])
        dx["deltas"] = dx["max"] - dx["min"]
        #dx["name"]=x.strip('.txt')
        ########### Extract channals ######

        ch0=dx[dx.channal == "Ch0:"]
        ch1=dx[dx.channal == "Ch1:"]
        ch2=dx[dx.channal == "Ch2:"]
        ch3=dx[dx.channal == "Ch3:"]
        ch4=dx[dx.channal == "Ch4:"]
        ch5=dx[dx.channal == "Ch5:"]
        ch6=dx[dx.channal == "Ch6:"]
        ch7=dx[dx.channal == "Ch7:"]
        ch8=dx[dx.channal == "Ch8:"]
        #print(ch0.head())
        ############ build the deltas #############
        delta0= ch0["max"] -  ch0["min"]
        lst['delta0']=delta0
        delta1= ch1["max"] -  ch1["min"]
        lst['delta1']=delta1
        delta2= ch2["max"] -  ch2["min"]
        lst['delta2']=delta2
        delta3= ch3["max"] -  ch3["min"]
        lst['delta3']=delta3
        delta4= ch4["max"] -  ch4["min"]
        lst['delta4']=delta4
        delta5= ch5["max"] -  ch5["min"]
        lst['delta5']=delta5
        delta6= ch6["max"] -  ch6["min"]
        lst['delta6']=delta6
        delta7= ch7["max"] -  ch7["min"]
        lst['delta7']=delta7
        delta8= ch8["max"] -  ch8["min"]
        lst['delta8']=delta8
        #print(delta0.head())
        ##################### format reset index

        df_new = pd.DataFrame.from_dict(lst)
        df_new=df_new.apply(lambda x: pd.Series(x.dropna().values))
        #print(df_new.head(3))
        #df_new.to_excel("dfnew.xlsx")
        dv=dx[dx.channal == "V:"]
        ########## connect the name of Measurement
        dv.index.name=x.strip('.txt')
        dx.index.name = x.strip('.txt')
        df_new.index.name=x.strip('.txt')

        dx=dx.reset_index()
        dv=dv.reset_index()
        df_new=df_new.reset_index()

        if not (dx.empty):
            a[x.strip('.txt')]=dx
        dv=dv.replace('V:', 'V:{}'.format(x.strip('.txt')))
        rows=df_new.shape[0]
        #print(rows)
        for row in range(rows):
            df_new["name"]=x.strip('.txt')
        #drop empty Dataframes
        if not (dx.empty):
            result=pd.concat([result,dx.append(pd.Series(name='V_Spacer',dtype='float'))],axis=1)
            verhaeltnis=pd.concat([verhaeltnis,dv],axis=1)
            new_df=result.append(verhaeltnis)
            deltas=pd.concat([deltas,df_new],axis=1)

        rows= deltas.shape[0]


#new_df=pd.concat([result,verhaeltnis])
#print(result.head())
#result.to_excel("result.xlsx")
verhaeltnis=verhaeltnis.drop(["max","deltas"], axis=1)
verhaeltnis=verhaeltnis.rename(columns={"min": "V"})
verhaeltnis.head()
#verhaeltnis.to_excel("verhaeltnis.xlsx")
# verhaeltnis.to_excel("FullData.xlsx",sheet_name='verhältnis')
#deltas.to_excel("deltas.xlsx")
# deltas.to_excel("FullData.xlsx",sheet_name='deltas')
new_df.to_excel("FullData.xls",sheet_name='fullData')
with pd.ExcelWriter('FullDataAll.xlsx') as writer:
    result.to_excel(writer, sheet_name='cleanValues')  
    new_df.to_excel(writer, sheet_name='cleanWithVerhältnis')
    deltas.to_excel(writer, sheet_name='deltas')
    verhaeltnis.to_excel(writer, sheet_name='verhaeltnis')
    new_df["spacer1"]=" "
    new_df["spacer2"]=" "
    merged_df=pd.merge(new_df,deltas.reset_index(),how='outer')
    merged_df.to_excel(writer, sheet_name='FullDataAll')


ModuleNotFoundError: No module named 'xlwt'

In [185]:
#print(new_df.head())
#print(deltas.head())
new_df["spacer1"]=" "
new_df["spacer2"]=" "
merged_df=pd.merge(new_df,deltas.reset_index(),how='outer')
merged_df.to_excel("merged.xlsx")
new_df.tail()

,fischer1_p33_6_hf8_fc6x_0_9km,channal,min,max,deltas,fischer1_p33_6_hf8_fc6x_4km,channal,min,max,deltas,...,min,max,deltas,madshus_base7_ts_blue_poin,channal,min,max,deltas,spacer1,spacer2
5,59.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1483.0,NaN,NaN,59.0,V:fischer1_p33_6_hf8_fc6x_4km,1845.0,NaN,NaN,...,1058.0,NaN,NaN,59.0,V:madshus_base7_ts_blue_poin,1029.0,NaN,NaN,,
6,69.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1734.0,NaN,NaN,69.0,V:fischer1_p33_6_hf8_fc6x_4km,1651.0,NaN,NaN,...,393.0,NaN,NaN,69.0,V:madshus_base7_ts_blue_poin,1043.0,NaN,NaN,,
7,79.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1547.0,NaN,NaN,79.0,V:fischer1_p33_6_hf8_fc6x_4km,1849.0,NaN,NaN,...,550.0,NaN,NaN,79.0,V:madshus_base7_ts_blue_poin,1037.0,NaN,NaN,,
8,89.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1735.0,NaN,NaN,89.0,V:fischer1_p33_6_hf8_fc6x_4km,1301.0,NaN,NaN,...,865.0,NaN,NaN,89.0,V:madshus_base7_ts_blue_poin,1064.0,NaN,NaN,,
9,99.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1549.0,NaN,NaN,99.0,V:fischer1_p33_6_hf8_fc6x_4km,1799.0,NaN,NaN,...,NaN,NaN,NaN,99.0,V:madshus_base7_ts_blue_poin,1004.0,NaN,NaN,,


In [182]:
deltas["spacer"]=" "
deltas.tail()

,fischer1_p33_6_hf8_fc6x_0_9km,delta0,delta1,delta2,delta3,delta4,delta5,delta6,delta7,delta8,...,delta1,delta2,delta3,delta4,delta5,delta6,delta7,delta8,name,spacer
5,5,1615185.0,891345.0,993157.0,661094.0,65904.0,2985950.0,288194.0,201955.0,832681.0,...,810495.0,989422.0,659191.0,54274.0,5657076.0,256138.0,163997.0,712343.0,madshus_base7_ts_blue_poin,
6,6,1634617.0,884525.0,949401.0,623054.0,65548.0,3513933.0,295455.0,213389.0,829571.0,...,812788.0,982172.0,659128.0,54998.0,5791345.0,255976.0,164437.0,710690.0,madshus_base7_ts_blue_poin,
7,7,1670156.0,907992.0,1006848.0,664693.0,67719.0,3724424.0,304490.0,225144.0,913982.0,...,822083.0,994893.0,667552.0,56311.0,5907071.0,254746.0,164004.0,707252.0,madshus_base7_ts_blue_poin,
8,8,1620760.0,876338.0,947155.0,617114.0,69606.0,4010816.0,256399.0,212102.0,705566.0,...,811023.0,991592.0,654957.0,54111.0,6034483.0,253999.0,165911.0,706846.0,madshus_base7_ts_blue_poin,
9,9,1575421.0,875885.0,938637.0,640912.0,65455.0,4085911.0,270144.0,207163.0,807122.0,...,811095.0,989232.0,663076.0,54215.0,6200550.0,253279.0,164666.0,706860.0,madshus_base7_ts_blue_poin,


In [157]:
new_df.shape

(111, 45)